In [ ]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import SNAPDataset
from torch_geometric.utils.convert import to_networkx
import numpy as np

dataset = Planetoid(root='', name='Citeseer')
data = dataset[0]
print(data)

In [ ]:
import networkx as nx

G = to_networkx(dataset[0], to_undirected = True)

In [ ]:
from community import community_louvain

partition = community_louvain.best_partition(G)

In [ ]:
communities = []
y_s = []

for part in partition:
  y_s.append(partition[part])
  if partition[part] not in communities:
    communities.append(partition[part])

communities.sort()

In [ ]:
import torch

y = torch.Tensor(y_s)
y = y.type(torch.LongTensor)

data.y = y

In [ ]:
from torch_geometric.transforms import RandomNodeSplit

split = "test_rest"

transform = RandomNodeSplit(split=split, num_val= 0)
data = transform(data)

In [ ]:
from torch_geometric.transforms import GCNNorm

norm = GCNNorm()
norm(data)

In [ ]:
from torch.nn import Linear
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv

class GCN(torch.nn.Module):
  def __init__(self):
    super(GCN, self).__init__()
    torch.manual_seed(42)

    self.conv1 = GCNConv(dataset.num_features, 16)
    self.conv4 = GCNConv(16, 4)
    self.dropout = torch.nn.Dropout(p=0.2)
    self.conv2 = GCNConv(4, len(communities))
    
  def forward(self, x, edge_index, edge_weight):
    h = self.conv1(x, edge_index, edge_weight)
    h = h.relu()
    
    h = self.conv4(h, edge_index, edge_weight)
    h = self.dropout(h)
    h = h.relu()

    h = self.conv2(h, edge_index, edge_weight)
    
    out = torch.nn.functional.log_softmax(h, dim=1)
    return out, h
    
model = GCN()

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train(data):
  model.train()
  optimizer.zero_grad()
  out, h = model(data.x, data.edge_index, data.edge_weight)
  loss = criterion(out[data.train_mask], y[data.train_mask])
  loss.backward()
  optimizer.step()
  return loss, h

def test(data):
  model.eval()
  out, h = model(data.x, data.edge_index, data.edge_weight)
  pred = out.argmax(dim=1)
  
  test_correct = pred[data.test_mask] == y[data.test_mask]

  train_correct = pred[data.train_mask] == y[data.train_mask]
  
  test_acc = int(test_correct.sum().item()) / int(data.test_mask.sum().item())  

  train_acc = int(train_correct.sum().item()) / int(data.train_mask.sum().item())  

  return test_acc, train_acc, pred

epochs = range(1, 1001)
losses = []
embeddings = []

pred = []
Ep = []
Test = []
Train = []

for epoch in epochs:
  loss, h = train(data)
  losses.append(loss)
  embeddings.append(h)

  Ep.append(epoch)
  pred = []

  test_acc, train_acc, pred = test(data)
  Train.append(train_acc)
  Test.append(test_acc)
  print(f"Epoch: {epoch}\tTest Accuracy: {test_acc:.4f} \tTrain Accuracy: {train_acc:.4f}")


In [ ]:
p = torch.Tensor.tolist(pred)
labels = torch.Tensor.tolist(data.y)

In [ ]:
from sklearn.metrics.cluster import normalized_mutual_info_score

normalized_mutual_info_score(labels, p)

In [ ]:
from sklearn.metrics import f1_score

f1_score(labels, p, average='micro')